<font color='darkblue' size=18>Zonas Basicas de Salud con python</font><img src="img/Mercator-projection.jpg" align='right'><img src="img/python-logo.png" align='right'>

### <font color='darkblue' size=12> Comunidad de Madrid</font>

#### Vamos a usar python y alguna de sus librerías para crear una representación gráfica de datos sobre un mapa.

##### Para ello necesitamos:

    1. Un mapa base.
    2. Shape de cada área del mapa.
    3. Datos a representar de cada área/shape.
    4. Asociar los datos de ZBS a cada shape.
    5. Mapa con los datos.

#### En este caso, serán datos de situación del COVID-19 en las Zonas Básicas de Salud de la Comunidad de Madrid.
### Necesitamos datos:
- Los shapes de las Zonas Básicas de Salud que forman el mapa de la Comunidad de Madrid.
- Un dato a representar en el mapa. He escogido la Tasa de Incidencia Acumulada en los últimos 14 días. Esta serie de datos se llamará **TIA-14**.

### Necesitamos librerías que nos permitan acceder a los datos, procesarlos y presentarlos de manera interactiva:
- Podemos usar datos en local ó acceder a ellos a traves de su url pública. En este caso será con ficheros locales. El ejemplo sin ficheros locales? **ZBS_Madrid_url.ipynb**
- Vamos a usar hvplot por su sencillez, está muy relacionado con pandas y permite acceder a holoviews y bokeh. Terminando con opciones interactivas.


<br><br>
### Empecemos importando las librerías que vamos a usar

In [ ]:
# # Las dos siguientes líneas de código nos permite usar el 100% del ancho de pantalla.
from IPython.display import display, HTML   
display(HTML("<style>.container { width:100% !important; }</style>"))


from IPython.core.interactiveshell import InteractiveShell  # Para mostrar más de una salida por celda en el notebook.

In [ ]:
import geopandas as gpd
import geoviews as gv
import cartopy.crs as ccrs
import pandas as pd
from bokeh.models import FixedTicker

gv.extension('bokeh')

<br><br>
## 1. Mapa Base.

    Será de Wikipedia. Hay otras opciones posibles de Mapa Base.

In [ ]:
Mapa_Base = gv.tile_sources.Wikipedia
Mapa_Base  # Ya tenemos Mapa Base y es interactivo!!

---
---
<br><br>
## 2. Colección de shapes de ZBS de la Comunidad de Madrid.


### - Cargamos en un GeoDataFrame los shapes y una columna extra pob_pad19. Los ficheros los proporciona la Comunidad de Madrid <img src="img/logo_comunidad.png" align='right' />
- La CAM proporciona un fichero zip con los shapes de las Zonas Básicas Sanitarias. [Datos de shapes](https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/f1837bd3-a835-4110-9bbf-fae06c99b56b/download/zonas_basicas_salud.zip)

[WEB Datos de Zonas Básicas de Madrid](https://datos.comunidad.madrid/catalogo/dataset/covid19_tia_zonas_basicas_salud)
- Podríamos acceder a estos ficheros directamente online ó descargarlo y usarlo como fichero local.
<br><br>
#### **PREVIAMENTE** he descargado esos ficheros a un subdirectorio llamado 'zonas_basicas_salud'
### Cargando los shapes

In [ ]:

gdf_CAM = gpd.read_file('zonas_basicas_salud') # Le he indicado el nombre del subdirectorio y geopandas se encarga de cargar lo necesario.

gdf_CAM_Mercator = gdf_CAM.to_crs(ccrs.GOOGLE_MERCATOR.proj4_init)  # Hay que adaptar las coordenadas de los shapes al estandar Mercator. Dejo los dos GeoDataFrame por si se quieren ver después.

In [ ]:
Mapa_ZBS = gv.Polygons(gdf_CAM_Mercator, vdims=['pob_pad19', 'zona_basic'], crs=ccrs.GOOGLE_MERCATOR)  
Mapa_ZBS   # Ya tenemos la capa de ZBS!!

### 2.1 Juntamos el Base y los Shapes de ZBS

In [ ]:
Mapa_Base * Mapa_ZBS  # Juntando Piezas.

----
----
<br><br>
## 3. Cargamos datos de COVID para cada ZBS

- La CAM proporciona un fichero csv con los datos COVID por Zona Básica Sanitaria. Se actualiza aproximadamente, una vez por semana, y suele ser los martes.  [Datos COVID-19 por Zona Básica Sanitaria](https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/43708c23-2b77-48fd-9986-fa97691a2d59/download/covid19_tia_zonas_basicas_salud_s.csv)

In [ ]:
# Cargamos los datos y los preparamos un poquito.

df_zonas_basicas = pd.read_csv('zonas_basicas_salud/covid19_tia_zonas_basicas_salud_s.csv', sep=';', encoding='iso8859_15', decimal=',', parse_dates=['fecha_informe'])

df_zonas_basicas.rename(columns={'tasa_incidencia_acumulada_ultimos_14dias':'TIA-14'}, inplace=True)  # Cambio el nombre de la columna por comodidad posterior.

df_zonas_basicas['codigo_geo']=df_zonas_basicas['codigo_geometria'].str.strip()  # Los codigos vienen con espacios por detrás. Se los quito y aprovecho para cambiar el nombre.
df_zonas_basicas['TIA-14']=df_zonas_basicas['TIA-14'].astype('int')
df_zonas_basicas.drop(columns='codigo_geometria', inplace=True)  # Ya no es necesaria esta columna.

df_zonas_basicas.head(3)

<br><br>
### 3.1 Filtramos los últimos datos del fichero

In [ ]:
#  En el fichero vienen los datos semanales desde el 2 de julio. Ahora, queremos el último dato de cada ZBS. 

df_zbs_ultimo_dia = df_zonas_basicas.loc[(df_zonas_basicas.fecha_informe==df_zonas_basicas.fecha_informe.max()),['TIA-14','codigo_geo','fecha_informe']]

df_zbs_ultimo_dia['fecha_informe']=df_zbs_ultimo_dia.fecha_informe.dt.strftime('%Y-%m-%d')

df_zbs_ultimo_dia.info(memory_usage='deep')

<br><br>
Vemos un DataFrame con 286 registros.

Uno por cada ZBS
Con 3 campos en cada registro.
- Codigo_geo:     Identifica a cada ZBS.
- fecha_informe:  fecha de los datos
- TIA-14:         Tasa de Incidencia Acumulada en los últimos 14 días.

In [ ]:
df_zbs_ultimo_dia.head(3)


----
----

<br><br>
## 4. Añadimos los datos de TIA-14 al GeoDataFrame

In [ ]:
gdf_CAM_COVID=gdf_CAM_Mercator.merge(df_zbs_ultimo_dia,  on='codigo_geo')
gdf_CAM_COVID.info(memory_usage='deep')

In [ ]:
Mapa_ZBS_COVID = gv.Polygons(gdf_CAM_COVID, vdims=['TIA-14',('zona_basic', 'Zona Básica de Salud'), ('fecha_informe', 'Fecha del Dato')], crs=ccrs.GOOGLE_MERCATOR, hover_cols=['pob_pad19'])
Mapa_ZBS_COVID

---
---
### 5. Mapa interactivo con los datos.

 - Juntamos Mapa Base y la capa Mapa_ZBS_COVID.
 - Tocamos algo las opciones para adaptar la visibilidad.

In [ ]:
colormap_COVID = ['#006837' , '#86cb66', '#ffffbf', '#f98e52', '#a50026', '#000000']
Niveles_COVID = [0,25,50, 150,250,400, 430]
Colorbar_ticks = FixedTicker(ticks=[  12.5, 25, 37.5, 50, 100, 150, 200, 250, 325, 400, 430])
Colorbar_labels =  {
            37.5: 'Riesgo Bajo',
            12.5: 'Nueva Normalidad',
            25:   '25',
            50:   '50',
            100: 'Riesgo Medio',
            150:  '150',
            200: 'Riesgo Alto',
            250:  '250',
            325: 'Riesgo Extremo',
            400:  '400',
            430: 'Riesgo IDA Extremo'
        }

# Mapa Base con Shapes de ZBS y con gradiente de color según TIA-14

In [ ]:
Mapa_Base * Mapa_ZBS_COVID.opts( color='TIA-14',  width=1000, height=900, tools=['hover'], alpha=0.7, color_levels=Niveles_COVID, clim=(0,430), cmap= colormap_COVID, colorbar=True, colorbar_opts={
        'major_label_overrides': Colorbar_labels,
        'major_label_text_align': 'left', 'ticker': Colorbar_ticks
    })

# Hacemos otro?

In [ ]:

Mapa_ZBS_COVID.opts(alpha=0.9, width=700,height=630)